In [18]:
import pandas as pd
import numpy as np


from neuralforecast import NeuralForecast
from neuralforecast.models import (
    NBEATS,
    NHITS,
    TFT)

from statsforecast.core import StatsForecast
from neuralforecast.auto import AutoMLP, AutoDeepAR, AutoNBEATS, AutoNHITS, AutoTFT , AutoDeepNPTS
from statsforecast.models import (
    Naive,
    SeasonalNaive,
    ARIMA,
    SimpleExponentialSmoothing,
    ETS,
    HistoricAverage,
    WindowAverage,
    AutoARIMA,
    AutoETS,
    AutoCES,
    AutoTheta
)

# Naive
# Naive season
# Seasonal Arima (0,1 1)( 0,1,1)

from mlforecast import MLForecast
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

import lightgbm as lgb
from mlforecast.target_transforms import Differences
from mlforecast.lag_transforms import ExpandingMean, RollingMean
from numba import njit
from window_ops.rolling import rolling_mean


import plotly.graph_objects as go


In [19]:
def evaluate_forecast(y_true, y_pred):
    return np.sqrt(np.mean((y_true.values - y_pred.values) ** 2))

def smape(y_true, y_pred):
    return 100 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)) / 2)

def rmsse(y_true, y_pred, train):
    naive_forecast = np.roll(train, 1)
    naive_forecast[0] = train[0]
    scale = np.mean((train - naive_forecast) ** 2)
    return np.sqrt(np.mean((y_true - y_pred) ** 2) / scale)

def calculate_errors(test, forecasts, train):

    df = pd.DataFrame()
    for col in forecasts.columns:
        error_dict = {}
        
        if col in ['ds', 'unique_id']:
            continue
        else:
            y_true = test['y'].values
            y_pred = forecasts[col].values
            error_dict.update({
                f'RMSE': evaluate_forecast(test['y'], forecasts[col]),
                f'SMAPE': smape(y_true, y_pred),
                f'RMSSE': rmsse(y_true, y_pred, train['y'].values)
            })
        df = pd.concat([pd.DataFrame(error_dict, index=[col]),df])
    return df

In [20]:

from statsforecast.utils import AirPassengersDF

# Load the AirPassengers dataset
df = AirPassengersDF

forecast_horizon = [24, 3,6,12]

# Train-test split
test_size_total = 24

train_size_total = len(df) - test_size_total
train_total, test_total = df[:train_size_total], df[train_size_total:]


df.to_csv('Air_passengers.csv')

In [21]:
import time
models = [
    Naive(),
    SeasonalNaive(12),
    ARIMA(order=[12,1,0]),
    ARIMA(order=[0,1,1], seasonal_order=[0,1,1], season_length=12 ,alias='SARIMA'),
    SimpleExponentialSmoothing(alpha=0.28),
    ETS(model='AAA',season_length=12),
    HistoricAverage(),
    WindowAverage(window_size=6),
    AutoARIMA(max_p=12),
    AutoETS(season_length=12),
    AutoCES(season_length=12,alias='AutoCES'),
    AutoTheta(season_length=12)
]

error_dfs = []

for model in models:
    for horizon in forecast_horizon:
        df_forecast_model = pd.DataFrame()
        total_train_time = 0
        combined_forecasts = pd.DataFrame()

        for start in range(0, test_size_total, horizon):
            end = start + horizon
            train_size = train_size_total + start
            train = df[:train_size]
            test = df[train_size:train_size + horizon]
            sf = StatsForecast(
                models=[model],
                freq='ME',
                n_jobs=-1,
            )

            start_time = time.time()
            forecasts_df = sf.forecast(df=train, h=horizon)
            train_time = time.time() - start_time

            forecasts_df['origin'] = train_size  # Track the forecast origin point
            combined_forecasts = pd.concat([combined_forecasts, forecasts_df])
            total_train_time += train_time
        
            
        # Calculate errors for the combined forecast
        combined_error_statsforecast = calculate_errors(test_total, combined_forecasts.drop(columns=['origin']), train_total)
        
        combined_error_statsforecast['Total_Train_Time'] = total_train_time
        combined_error_statsforecast['Horizon'] = horizon
        combined_error_statsforecast['Model'] = str(model)
        error_dfs.append(combined_error_statsforecast)

# Combine all errors into a single DataFrame
Error_statstical = pd.concat(error_dfs).reset_index(drop=True)

c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\statsforecast\models.py:880: FutureWarning: `ETS` will be deprecated in future versions of `StatsForecast`. Please use `AutoETS` instead.
  ETS._warn()
c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\statsforecast\core.py:485: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\statsforecast\core.py:485: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\statsforecast\core.py:485: FutureWarning: In a future version the predictions will have the id

In [22]:
@njit
def rolling_mean_12(x):
    return rolling_mean(x, window_size=12)


def month_index(times):
    return times.month

In [23]:
lgb_params = {'verbosity': -1,'num_leaves': 512,}

catboost_params ={'subsample': 0.6 , 'iterations': 50, 'depth': 5, 'verbose':0}

xgboost_params ={'verbosity':0, 'max_depth':5 , 'subsample': 0.6}

randomforest_params = {'verbose': 0, 'max_depth': 5}
models={
        'LightGBM': lgb.LGBMRegressor(**lgb_params),
        'CatBoost': CatBoostRegressor(**catboost_params),
        'XgBoost': XGBRegressor(**xgboost_params),
        'RandomForest': RandomForestRegressor(**randomforest_params)
    }


In [24]:

lgb_params = {'verbosity': -1,'num_leaves': 512,}

catboost_params ={'subsample': 0.6 , 'iterations': 50, 'depth': 5, 'verbose':0}

xgboost_params ={'verbosity':0, 'max_depth':5 , 'subsample': 0.6}

randomforest_params = {'verbose': 0, 'max_depth': 5}
models={
        'LightGBM': lgb.LGBMRegressor(**lgb_params),
        'CatBoost': CatBoostRegressor(**catboost_params),
        'XgBoost': XGBRegressor(**xgboost_params),
        'RandomForest': RandomForestRegressor(**randomforest_params)
    }

for alias,model in models.items():
    for horizon in forecast_horizon:
        df_forecast_model = pd.DataFrame()
        total_train_time = 0
        combined_forecasts = pd.DataFrame()

        for start in range(0, test_size_total, horizon):
            end = start + horizon
            train_size = train_size_total + start
            train = df[:train_size]
            test = df[train_size:train_size + horizon]
            
            fcst = MLForecast(
                models = {alias:model,},
                freq="ME",
                target_transforms=[Differences([12])],    
                lags= [1,2,3,4,11,12],
                lag_transforms={
                    1: [ExpandingMean()],
                    12: [RollingMean(window_size=12), rolling_mean_12],
                },
                date_features=[month_index],
            )

            start_time = time.time()
            prep = fcst.preprocess(train)
            fcst.fit(train)
            forecasts_df = fcst.predict(h=horizon)
            train_time = time.time() - start_time

            forecasts_df['origin'] = train_size  # Track the forecast origin point
            combined_forecasts = pd.concat([combined_forecasts, forecasts_df])
            total_train_time += train_time
        
            
        # Calculate errors for the combined forecast
        combined_error_statsforecast = calculate_errors(test_total, combined_forecasts.drop(columns=['origin']), train_total)
        
        combined_error_statsforecast['Total_Train_Time'] = total_train_time
        combined_error_statsforecast['Horizon'] = horizon
        combined_error_statsforecast['Model'] = alias
        error_dfs.append(combined_error_statsforecast)

# Combine all errors into a single DataFrame
Error_Tree = pd.concat(error_dfs).reset_index(drop=True)

In [25]:
Error_Tree

,RMSE,SMAPE,RMSSE,Total_Train_Time,Horizon,Model
0,137.328985,6.937759,4.801815,0.000000,24,Naive
1,62.552311,2.837492,2.187191,0.030835,3,Naive
2,77.368167,3.688349,2.705239,0.015018,6,Naive
3,108.203127,4.838893,3.783407,0.016149,12,Naive
4,76.994589,4.253156,2.692176,0.000000,24,SeasonalNaive
...,...,...,...,...,...,...
59,16.963066,0.835843,0.593127,0.248883,12,XgBoost
60,37.899424,2.002011,1.325183,0.268242,24,RandomForest
61,23.136272,1.055906,0.808978,1.217131,3,RandomForest
62,22.665763,1.150651,0.792526,0.847048,6,RandomForest


In [26]:
from ray import tune
neural_models = [
    NBEATS(input_size=2 * test_size_total, h=test_size_total, max_steps=50),
    NHITS(input_size=2 * test_size_total, h=test_size_total, max_steps=50),
    AutoMLP(config=dict(max_steps=20, input_size=tune.choice([3 * test_size_total]), learning_rate=tune.choice([1e-3])), h=test_size_total, num_samples=1, cpus=3),
    AutoDeepAR(config=dict(max_steps=5, input_size=tune.choice([3 * test_size_total]), learning_rate=tune.choice([1e-3])), h=test_size_total, num_samples=1, cpus=3),
    AutoNBEATS(config=dict(max_steps=20, input_size=tune.choice([3 * test_size_total]), learning_rate=tune.choice([1e-3])), h=test_size_total, num_samples=1, cpus=3),
    AutoNHITS(config=dict(max_steps=20, input_size=tune.choice([3 * test_size_total]), learning_rate=tune.choice([1e-3])), h=test_size_total, num_samples=1, cpus=3),
    AutoTFT(config=dict(max_steps=20, input_size=tune.choice([3 * test_size_total]), learning_rate=tune.choice([1e-3])), h=test_size_total, num_samples=1, cpus=3)
]

# #AutoMLP, AutoDeepAR, AutoNBEATS, AutoNHITS, AutoTFT , AutoDeepNPTS
# nf = NeuralForecast(models=models, freq='M')

# nf.fit(df=train,)
# forecasts_df_neural = nf.predict().reset_index()

# error_neural = calculate_errors(test,forecasts_df_neural,train)


Seed set to 1
Seed set to 1
c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\pytorch_lightning\utilities\parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\pytorch_lightning\utilities\parsing.py:199: Attribute 'valid_loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['valid_loss'])`.


In [27]:
# error_dfs=[]
for model in neural_models:
    print(f"------------{model}------------")
    for horizon in forecast_horizon:
        total_train_time = 0
        combined_forecasts = pd.DataFrame()

        for start in range(0, test_size_total, horizon):
            end = start + horizon
            train_size = train_size_total + start
            train = df[:train_size]
            test = df[train_size:train_size + horizon]
            
            nf = NeuralForecast(models=[model], freq='ME')

            start_time = time.time()
            nf.fit(df=train)
            forecasts_df_neural = nf.predict().reset_index()
            train_time = time.time() - start_time
            forecasts_df_neural = forecasts_df_neural[:horizon]
            forecasts_df_neural['origin'] = train_size  # Track the forecast origin point
            combined_forecasts = pd.concat([combined_forecasts, forecasts_df_neural])
            total_train_time += train_time

        # Calculate errors for the combined forecast
        combined_error_statsforecast = calculate_errors(test_total, combined_forecasts.drop(columns=['origin']), train_total)
        combined_error_statsforecast['Total_Train_Time'] = total_train_time
        combined_error_statsforecast['Horizon'] = horizon
        combined_error_statsforecast['Model'] = type(model).__name__
        error_dfs.append(combined_error_statsforecast)

# Combine all errors into a single DataFrame
all_errors = pd.concat(error_dfs).reset_index(drop=True)
print(all_errors)

(_train_tune pid=25440) c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\ray\tune\integration\pytorch_lightning.py:198: `ray.tune.integration.pytorch_lightning.TuneReportCallback` is deprecated. Use `ray.tune.integration.pytorch_lightning.TuneReportCheckpointCallback` instead.
(_train_tune pid=25440) Seed set to 1
(_train_tune pid=25440) c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\torch\nn\init.py:452: UserWarning: Initializing zero-element tensors is a no-op
(_train_tune pid=25440)   warnings.warn("Initializing zero-element tensors is a no-op")
(_train_tune pid=25440) c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\neuralforecast\common\_base_model.py:208: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
(_train_tune pid=25440)   warnings.warn(
(_train_tune pid=25440) GPU available: False, used: False
(_train_tune pid=25440) TPU available: False, using: 0 TPU cores
(_train

Epoch 19:   0%|          | 0/1 [00:00<?, ?it/s, v_num=0, train_loss_step=1.180, train_loss_epoch=1.180]        


2024-07-07 02:39:32,216	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-07-07 02:39:32,220	INFO tune.py:1007 -- Wrote the latest version of all result files and experiment state to 'C:/Users/91976/ray_results/_train_tune_2024-07-07_02-37-19' in 0.0111s.
Seed set to 1
c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\torch\nn\init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op

Epoch 19: 100%|██████████| 1/1 [00:06<00:00,  0.16it/s, v_num=0, train_loss_step=1.180, train_loss_epoch=1.180]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 19: 100%|██████████| 1/1 [00:06<00:00,  0.16it/s, v_num=0, train_loss_step=1.180, train_loss_epoch=1.180, valid_loss=67.40]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=20` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\neuralforecast\core.py:196: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


           RMSE     SMAPE     RMSSE  Total_Train_Time  Horizon          Model
0    137.328985  6.937759  4.801815          0.000000       24          Naive
1     62.552311  2.837492  2.187191          0.030835        3          Naive
2     77.368167  3.688349  2.705239          0.015018        6          Naive
3    108.203127  4.838893  3.783407          0.016149       12          Naive
4     76.994589  4.253156  2.692176          0.000000       24  SeasonalNaive
..          ...       ...       ...               ...      ...            ...
107   49.404549  2.294722  1.727469         36.422247       12      AutoNHITS
108   82.039027  3.978888  2.868559        259.732192       24        AutoTFT
109   83.678559  3.973067  2.925886       2060.273209        3        AutoTFT
110   88.133577  4.244692  3.081660       1033.758155        6        AutoTFT
111   89.099215  4.338557  3.115424        501.649775       12        AutoTFT

[112 rows x 6 columns]


In [28]:
from nixtla import NixtlaClient
nixtla_client = NixtlaClient(
    api_key = 'nixtla-tok-BWWtvgUP9FLtzerA90xyzXPvRUoZvA0OYYp5cuSI7NZUyApQjlINlF8dAyYXqDyxWlTlCOg7jXHWJV4o'
)

In [29]:
# error_dfs = []
for horizon in forecast_horizon:
    df_forecast_model = pd.DataFrame()
    total_train_time = 0
    combined_forecasts = pd.DataFrame()

    for start in range(0, test_size_total, horizon):
        end = start + horizon
        train_size = train_size_total + start
        train = df[:train_size]
        test = df[train_size:train_size + horizon]
        sf = StatsForecast(
            models=[model],
            freq='ME',
            n_jobs=-1,
        )

        start_time = time.time()
        forecasts_df = nixtla_client.forecast(df=train, h=horizon, freq='M', time_col='ds', target_col='y')
        train_time = time.time() - start_time

        forecasts_df['origin'] = train_size  # Track the forecast origin point
        combined_forecasts = pd.concat([combined_forecasts, forecasts_df])
        total_train_time += train_time
    
        
    # Calculate errors for the combined forecast
    combined_error_statsforecast = calculate_errors(test_total, combined_forecasts.drop(columns=['origin']), train_total)
    
    combined_error_statsforecast['Total_Train_Time'] = total_train_time
    combined_error_statsforecast['Horizon'] = horizon
    combined_error_statsforecast['Model'] = "TimeGPT"
    error_dfs.append(combined_error_statsforecast)

# Combine all errors into a single DataFrame
Error_TimeGPT = pd.concat(error_dfs).reset_index(drop=True)

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: ME
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: ME
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: ME
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: ME
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.

In [35]:

# Sort the DataFrame by the 'Error' column
sorted_df = Error_TimeGPT.sort_values(by='RMSSE')



sorted_df.to_csv('results.csv')


In [36]:
sf.plot(df,forecasts_df,engine='plotly')

In [39]:
combined_error_statsforecast.to_csv('forecast_timegpt.csv')

In [32]:
sf.plot(df,forecasts_df_tree.reset_index())


NameError: name 'forecasts_df_tree' is not defined

In [ ]:
sf.plot(df,forecasts_df_neural,engine='plotly')


In [ ]:
sf.plot(df,forecasts_df_timegpt)
